# HW4: Fine-tuning BERT for entity labeling
This notebook contains starter code for finetuning a BERT-style model for the task of entity recognition. 

In [ ]:
import time

import torch
from torch import nn
from torch.utils.data import DataLoader, Subset #random_split
import numpy as np

torch.random.manual_seed(8942764)
torch.cuda.manual_seed(8942764)
np.random.seed(8942764)

device = 'cuda:1'

# if you have an Apple Silicon machine with a GPU, use the following setting
# this should about 3-4 times faster that running it on just CPU
# device = 'mps'

# If you will use a cpu, this is the setting
# device = 'cpu'

# Note that in handin.py these next two lines will need to be removed
# if you are going run this on your personal machine you will need to install
# these locally in the shell/terminal.

# !pip install protobuf==3.20.2
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install seqeval

from transformers import AutoTokenizer, BertModel, DataCollatorForTokenClassification

import evaluate

In [ ]:
device = 'cuda:0'

In [ ]:
# Load the dataset
from datasets import ClassLabel, Sequence, load_dataset

data_splits = load_dataset('json', data_files={'train': 'dinos_and_deities_train_bio.jsonl', 'dev': 'dinos_and_deities_dev_bio_sm.jsonl'})

label_names_fname = "export (2).csv"
labels_int2str = []
with open(label_names_fname) as f:
    labels_int2str = f.read().split()
print(f"Labels: {labels_int2str}")
print(f"{len(labels_int2str)}")
labels_str2int = {l: i for i, l in enumerate(labels_int2str)}

data_splits.cast_column("ner_tags", Sequence(ClassLabel(names=labels_int2str)))
print(data_splits)

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# If you want you can look at some sample data items
print(data_splits["train"][8])
print(data_splits["dev"][5])

{'para_index': 0, 'title': 'Myersiohyla liliae', 'doc_id': 'Myersiohyla liliae-0', 'content': 'Myersiohyla liliae is a species of frogs in the family Hylidae. It is endemic to the Pacaraima Mountains in Guyana and known from the region of its type locality in the Kaieteur National Park and from Imbaimadai. The species is dedicated to the daughter of its describer, Lili Kok.', 'page_id': '28259031', 'id': 'Ud-DXIcB1INCf0UyAseC', 'tokens': ['Myersiohyla', 'liliae', 'is', 'a', 'species', 'of', 'frogs', 'in', 'the', 'family', 'Hylidae.', 'It', 'is', 'endemic', 'to', 'the', 'Pacaraima', 'Mountains', 'in', 'Guyana', 'and', 'known', 'from', 'the', 'region', 'of', 'its', 'type', 'locality', 'in', 'the', 'Kaieteur', 'National', 'Park', 'and', 'from', 'Imbaimadai.', 'The', 'species', 'is', 'dedicated', 'to', 'the', 'daughter', 'of', 'its', 'describer,', 'Lili', 'Kok.'], 'ner_strings': ['B-Aquatic_animal', 'I-Aquatic_animal', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
# This dataset is split into a train, validation and test set, and each token has a label.
# Data from the dataset can generally be accessed like a Python dict.
# Note that 
print(data_splits['train'].features)

# Print the original sentence (which is whitespace tokenized).
example_input_tokens = data_splits['train'][8]['tokens']
print(f"Original tokens: {example_input_tokens}")

# Print the labels of the sentence.
example_ner_labels = data_splits['train'][8]['ner_tags']
print(f"NER labels: {example_ner_labels}")

# Map integer to string labels for the sentence
example_mapped_labels = [labels_int2str[l] for l in example_ner_labels]
print(f'Labels: {example_mapped_labels}')

# Print the sentence split into tokens.
example_tokenized = tokenizer(example_input_tokens, is_split_into_words=True)
print('BERT Tokenized: ', example_tokenized.tokens())

# Print the number of tokens in the vocabulary
print(f'Vocab size: {tokenizer.vocab_size}')

# # Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(example_tokenized.tokens()))

# Visualizing Mapping 
print(example_tokenized.word_ids())

{'para_index': Value(dtype='int64', id=None), 'title': Value(dtype='string', id=None), 'doc_id': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None), 'page_id': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_strings': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
Original tokens: ['Myersiohyla', 'liliae', 'is', 'a', 'species', 'of', 'frogs', 'in', 'the', 'family', 'Hylidae.', 'It', 'is', 'endemic', 'to', 'the', 'Pacaraima', 'Mountains', 'in', 'Guyana', 'and', 'known', 'from', 'the', 'region', 'of', 'its', 'type', 'locality', 'in', 'the', 'Kaieteur', 'National', 'Park', 'and', 'from', 'Imbaimadai.', 'The', 'species', 'is', 'dedicated', 'to', 'the', 'daughter', 'of', 'its', 'describer,', 'Lili', 'Kok.']
NER labels: [5, 0, 12, 12, 12, 12, 12, 12, 12, 12, 12,

In [ ]:
# Function that uses that along with the original labels to get the new set of labels
# for each BERT-tokenized token
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            str_label = labels_int2str[label]
            if str_label[0] == 'B':
                new_str_label = 'I' + str_label[1:]
                label = labels_str2int[new_str_label]
            new_labels.append(label)

    return new_labels

In [ ]:
tokenizer_aligned_labels = align_labels_with_tokens(example_ner_labels, example_tokenized.word_ids())
print(f'Aligned labels: {tokenizer_aligned_labels}')
print(f'Mapped aligned labels: {[labels_int2str[l] if l >= 0 else "_" for l in tokenizer_aligned_labels]}')

Aligned labels: [-100, 5, 0, 0, 0, 0, 0, 0, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]
Mapped aligned labels: ['_', 'B-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '_']


In [ ]:
# Checking Function
aligned_labels = align_labels_with_tokens(example_ner_labels, example_tokenized.word_ids())
print(f"Tokens: {example_tokenized.tokens()}")
print(f"Aligned labels: {[labels_int2str[l] if l >= 0 else '_' for l in aligned_labels]}")

Tokens: ['[CLS]', 'Myers', '##io', '##hyl', '##a', 'l', '##ilia', '##e', 'is', 'a', 'species', 'of', 'frogs', 'in', 'the', 'family', 'H', '##yl', '##idae', '.', 'It', 'is', 'endemic', 'to', 'the', 'Pac', '##ara', '##ima', 'Mountains', 'in', 'Guyana', 'and', 'known', 'from', 'the', 'region', 'of', 'its', 'type', 'locality', 'in', 'the', 'Kai', '##ete', '##ur', 'National', 'Park', 'and', 'from', 'I', '##mba', '##ima', '##dai', '.', 'The', 'species', 'is', 'dedicated', 'to', 'the', 'daughter', 'of', 'its', 'describe', '##r', ',', 'Lil', '##i', 'Ko', '##k', '.', '[SEP]']
Aligned labels: ['_', 'B-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'I-Aquatic_animal', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [ ]:
# Need to get the whole dataset into this format, so need to write a fn
# we can apply across all examples using Dataset.map.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
# Now we can apply that fn to tokenize all the data
tokenized_data_splits = data_splits.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=data_splits["train"].column_names,
)

In [ ]:
# Testing batcher
print("Examples:")
for i in range(2):
    print(tokenized_data_splits["train"][i]["labels"])

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
batch = data_collator([tokenized_data_splits["train"][i] for i in range(2)])

Examples:
[-100, 9, 4, 4, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 9, 4, 4, 4, 4, 4, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]
[-100, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,

In [ ]:
# Evaluation: we can use the seqeval library to handle calculating span-level precision, recall and F1
metric = evaluate.load("seqeval")

labels = data_splits["train"][0]["ner_tags"]
labels = [labels_int2str[i] for i in labels]
print(labels)

# Make a small change and see how it impacts the score
predictions = labels.copy()
predictions[0] = "O"
predictions[1]= "B-Animal"
metric.compute(predictions=[predictions], references=[labels])

['B-Cretaceous_dinosaur', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Cretaceous_dinosaur', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Animal': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'Cretaceous_dinosaur': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'overall_precision': 0.5,
 'overall_recall': 0.5,
 'overall_f1': 0.5,
 'overall_accuracy': 0.9809523809523809}

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
#imported tools for calculating metrics
# This code runs evaluation on test data.
@torch.no_grad()
def run_eval(model, dataset, batch_size, device, collate_fn):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    all_preds = []
    all_true = []
    total_loss = 0
    # Metric for seqeval
    metric = evaluate.load("seqeval")

    for batch in dataloader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs)
        loss = loss_fn(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        total_loss += loss.item()

        preds = outputs.argmax(dim=-1).detach().cpu().numpy()
        true_labels = labels.detach().cpu().numpy()

        # Converting predictions to label strings and aligning them
        for i in range(len(true_labels)):
            label_seq = true_labels[i]
            pred_seq = preds[i]

            aligned_labels = [labels_int2str[label] for label in label_seq if label != -100]
            aligned_preds = [labels_int2str[pred] for idx, pred in enumerate(pred_seq) if label_seq[idx] != -100]
            print(aligned_preds)
            all_true.append(aligned_labels)
            all_preds.append(aligned_preds)

    avg_loss = total_loss / len(dataloader)

    # Compute metrics using seqeval
    results = metric.compute(predictions=all_preds, references=all_true)

    return avg_loss, results


In [ ]:
# This code trains the model and evaluates it on test data. It should print
# progress messages during training indicating loss, accuracy and training speed.

def train(model, train_dataset, val_dataset, num_epochs, batch_size, optimizer_cls, lr, weight_decay, device, collate_fn=None, log_every=100):
    model = model.to(device)
    dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    if optimizer_cls == 'SGD':
      optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_cls == 'Adam':
      optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_cls == 'AdamW':
      optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)


    #combined all lists for sake of tracking them in an easier manner, i.e. for run_eval
    metrics_storage = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(num_epochs):
        model.train(True)
        total_loss = 0
        for i, batch in enumerate(dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            loss = loss_fn(outputs.view(-1, model.classifier.out_features), batch['labels'].view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            if (i + 1) % log_every == 0:
                print(f'Epoch {epoch + 1}/{num_epochs}, Step {i + 1}, Loss: {loss.item():.4f}')

        avg_train_loss = total_loss / len(dataloader)
        metrics_storage['train_loss'].append(avg_train_loss)

        #took out run_eval for the sake of running train faster
        if val_dataset is not None:
            model.eval()
            with torch.no_grad():
                total_val_loss = 0
                for batch in DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn):
                    batch = {k: v.to(device) for k, v in batch.items()}
                    outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
                    val_loss = loss_fn(outputs.view(-1, model.classifier.out_features), batch['labels'].view(-1))
                    total_val_loss += val_loss.item()
                avg_val_loss = total_val_loss / len(DataLoader(val_dataset, batch_size=batch_size))
                metrics_storage['val_loss'].append(avg_val_loss)

    return model, metrics_storage

In [ ]:
# This code defines the token classification class using BERT.
# The classifier is defined on top of the final layer of BERT.

class BertForTokenClassification(nn.Module):
  def __init__(self, bert_pretrained_config_name, num_classes, freeze_bert=False, dropout_prob=0.1):
    '''
    BERT with a classification MLP
    args:
    - bert_pretrained_config_name (str): model name from huggingface hub
    - num_classes (int): number of classes in the classification task
    - freeze_bert (bool): [default False] If true gradients are not computed for
                          BERT's parameters.
    - dropout_prob (float): [default 0.1] probability of dropping each activation.
    '''
    super(BertForTokenClassification, self).__init__()
    self.bert = BertModel.from_pretrained(bert_pretrained_config_name)
    self.bert.requires_grad_(not freeze_bert)
    self.dropout = nn.Dropout(dropout_prob)
    self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

  def forward(self, input_ids, attention_mask=None, token_type_ids=None):
    outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    sequence_output = outputs[0]
    sequence_output = self.dropout(sequence_output)
    logits = self.classifier(sequence_output)
    return logits


In [ ]:
device = 'cuda:0'

In [ ]:
# Fine-tuning happens here****

# At the end of each epoch, you also see validation loss and validation accuracy.
# Change the device as described above if you will not be using a GPU

# Set the random seed(s) for reproducability
torch.random.manual_seed(8942764)
torch.cuda.manual_seed(8942764)
np.random.seed(8942764)

# Make sure this is the same as you use for tokenization!
bert_model = 'bert-base-cased'

num_labels = len(labels_int2str)
print(f"Num labels: {num_labels}")

# conll hyperparams
# multiply your learning rate by k when using batch size of kN
lr = 4*2e-5
weight_decay = 0.01
epochs = 5
batch_size = 32
dropout_prob = 0.2
freeze_bert = False

bert_cls = BertForTokenClassification(bert_model, num_labels, dropout_prob=dropout_prob, freeze_bert=freeze_bert)

print(f'Trainable parameters: {sum([p.numel() for p in bert_cls.parameters() if p.requires_grad])}\n')

# Flag for setting "debug" mode. Set debug to False for full training.
debug = False

# Sample a subset of the training data for faster iteration in debug mode
subset_size = 1000
subset_indices = torch.randperm(len(tokenized_data_splits['train']))[:subset_size]
train_subset = Subset(tokenized_data_splits['train'], subset_indices)

bert_cls, bert_cls_logs = train(bert_cls, tokenized_data_splits['train'] if not debug else train_subset, tokenized_data_splits['dev'],
                                num_epochs=epochs, batch_size=batch_size, optimizer_cls='AdamW',
                                lr=lr, weight_decay=weight_decay, device=device,
                                collate_fn=data_collator, log_every=10 if debug else 100)

# Final eval
final_loss, final_metrics = run_eval(bert_cls, tokenized_data_splits['dev'], batch_size=32, device=device, collate_fn=data_collator)
final_acc = final_metrics['overall_accuracy']
final_p = final_metrics['overall_precision']
final_r = final_metrics['overall_recall']
final_f1 = final_metrics['overall_f1']
print(f'\nFinal Loss: {final_loss:.3e}\t Final Accuracy: {final_acc:.3f}\t dev_p:{final_p:.3f}\t dev_r:{final_r:.3f}\t dev_f1:{final_f1:.3f}')


Num labels: 13
Trainable parameters: 108320269

['O', 'O', 'O', 'O', 'O', 'B-Goddess', 'I-Goddess', 'I-Goddess', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Goddess', 'I-Goddess', 'I-Goddess', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [ ]:

# Final eval
final_loss, final_metrics = run_eval(bert_cls, tokenized_data_splits['dev'], batch_size=32, device='cuda:0', collate_fn=data_collator)
final_acc = final_metrics['overall_accuracy']
final_p = final_metrics['overall_precision']
final_r = final_metrics['overall_recall']
final_f1 = final_metrics['overall_f1']
print(f'\nFinal Loss: {final_loss:.3e}\t Final Accuracy: {final_acc:.3f}\t dev_p:{final_p:.3f}\t dev_r:{final_r:.3f}\t dev_f1:{final_f1:.3f}')




['O', 'O', 'O', 'O', 'O', 'B-Goddess', 'I-Goddess', 'I-Goddess', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'I-Aquatic_mammal', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Goddess', 'I-Goddess', 'I-Goddess', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Aquatic_mammal', 'I-Aquatic_mammal', '

In [ ]:
torch.save(bert_cls.state_dict(), 'bert.pt')


In [ ]:
# Make sure this is the same as you use for tokenization
bert_model = 'bert-base-cased'

num_labels = len(labels_int2str)
print(f"Num labels: {num_labels}")

# conll hyperparams
# multiply your learning rate by k when using batch size of kN
lr = 4*2e-5
weight_decay = 0.01
epochs = 5
batch_size = 16
dropout_prob = 0.2
freeze_bert = False
bert_cls=BertForTokenClassification(bert_model, num_labels, dropout_prob=dropout_prob, freeze_bert=freeze_bert)
bert_cls.load_state_dict(torch.load('bert.pt'))
bert_cls = bert_cls.to('cuda')

Num labels: 13


In [ ]:
import json
# Create an empty JSON file to store results
with open('test_predictions_llm_baseline.json', 'w') as f:
    json.dump([], f)  # Start with an empty list

with open('test_predictions_llm_baseline.json', 'w') as f:
      json.dump(all_predictions, f, indent=4)

NameError: name 'all_predictions' is not defined